In [1]:
from openai import OpenAI

In [2]:
client = OpenAI()

## Q1 Running Elastic

- Getting data
- Indexing data
- Searching
- Filtering
- Build a prompt

### Getting data

In [3]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Indexing data

In [5]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

index_name = "hw"
es = Elasticsearch('http://localhost:9200')  # Connect to your local Elasticsearch instance

# create Index with es
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} # keyword for filtering
        }
    }
}

index_name = 'hw-llm'
es.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'hw-llm'})

In [6]:
for doc in tqdm(documents):
    es.index(index=index_name, body=doc) 

100%|██████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:22<00:00, 41.97it/s]


### Searching

In [7]:
query = "How do I execute a command in a running docker container?"
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
            }
        }
    }

response = es.search(index=index_name, body=search_query)

In [8]:
top_score = response['hits']['hits'][0]['_score']

In [9]:
top_score

84.050095

### Q4 Filtering

In [23]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], # question are three times more important
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": 'machine-learning-zoomcamp'
                }
            }
        }
    }
}

In [24]:
response = es.search(index=index_name, body=search_query)

result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [25]:
result_docs[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

# Q5 Building a prompt

In [26]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()

    context = ""
                    
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [27]:
query

'How do I execute a command in a running docker container?'

In [28]:
prompt_results = build_prompt(query, result_docs)

In [30]:
len(prompt_results)

1487

# Q6 Tokens

In [31]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 548.4 kB/s eta 0:00:00MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.2 MB/s eta 0:00:00m eta 0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 19.2 MB/s eta 0:00:00m eta 0:00:01


In [33]:
import tiktoken

In [34]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [36]:
token_prompt = encoding.encode(prompt_results)

In [37]:
len(token_prompt)

329

# Bonus Send to OpenAI

In [39]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [42]:
answer_results = llm(prompt_results)

In [46]:
answer_results

'To execute a command in a running Docker container, you can use the `docker exec` command. Here is the step-by-step process:\n\n1. **List Running Containers**: First, identify the running container by listing all the active containers.\n    ```sh\n    docker ps\n    ```\n\n2. **Execute Command**: Once you have identified the `container-id` of the container you want to interact with, you can execute a command inside that container. For example, to start a bash session, you can use:\n    ```sh\n    docker exec -it <container-id> bash\n    ```\n\nReplace `<container-id>` with the actual ID of the running container.\n\nThis allows you to run commands inside the context of an already running Docker container.'

In [44]:
token_answer = encoding.encode(answer_results)

In [54]:
len(token_prompt) * 0.005 + len(token_answer) * 0.015

3.9699999999999998